In [ ]:
######################## THE CODE FOR HW 4 is below ####################################

In [1]:
import pandas as pd

In [3]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)
df = df.iloc[:300]
print(df.head())

                                          answer_llm  \
0  You can sign up for the course by visiting the...   
1  You can sign up using the link provided in the...   
2  Yes, there is an FAQ for the Machine Learning ...   
3  The context does not provide any specific info...   
4  To structure your questions and answers for th...   

                                         answer_orig  document  \
0  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
1  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
2  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
3  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
4  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   

                                            question  \
0                Where can I sign up for the course?   
1                 Can you provide a link to sign up?   
2  Is there an FAQ for this Machine Learning course?   
3  Does this course have a GitHub reposito

In [4]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/home/dheeraj/miniconda3/envs/llm_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [6]:
answer_llm = df.iloc[0].answer_llm
print(answer_llm)
print(embedding_model.encode(answer_llm))

You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).
[-4.22446579e-01 -2.24856094e-01 -3.24058473e-01 -2.84758508e-01
  7.25637982e-03  1.01186477e-01  1.03716679e-01 -1.89983442e-01
 -2.80597862e-02  2.71588922e-01 -1.15337484e-01  1.14666171e-01
 -8.49585980e-02  3.32365513e-01  5.52722663e-02 -2.22195774e-01
 -1.42540708e-01  1.02519087e-01 -1.52333796e-01 -2.02912658e-01
  1.98424123e-02  8.38147700e-02 -5.68632305e-01  2.32842825e-02
 -1.67292669e-01 -2.39256859e-01 -8.05463567e-02  2.57082582e-02
 -8.15465227e-02 -7.39290714e-02 -2.61549950e-01  1.92575473e-02
  3.22909385e-01  1.90357044e-01 -9.34726413e-05 -2.13165492e-01
  2.88941171e-02 -1.79531835e-02 -5.92757724e-02  1.99918330e-01
 -4.75172400e-02  1.71634063e-01 -2.45916881e-02 -9.38061625e-02
 -3.57002705e-01  1.33263960e-01  1.94045797e-01 -1.18530668e-01
  4.56915230e-01  1.47728100e-01  3.35945100e-01 -1.86959475e-01
  2.45955020e-01 -1.07774369e-01 -1.8085964

In [18]:
import numpy as np
evaluations = []
for i in range(df.shape[0]):
    dot_ = np.dot(embedding_model.encode(df.iloc[i].answer_llm),embedding_model.encode(df.iloc[i].answer_orig))
    evaluations.append(dot_)    

In [19]:
print(pd.Series(evaluations).describe())

count    300.000000
mean      27.495996
std        6.384742
min        4.547924
25%       24.307847
50%       28.336872
75%       31.674312
max       39.476013
dtype: float64


In [21]:
def norm_func(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [22]:
import numpy as np
evaluations_norm = []
for i in range(df.shape[0]):
    v1 = norm_func(embedding_model.encode(df.iloc[i].answer_llm))
    v2 = norm_func(embedding_model.encode(df.iloc[i].answer_orig))
    dot_ = np.dot(v1,v2)
    evaluations_norm.append(dot_)    

In [23]:
print(pd.Series(evaluations_norm).describe())

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
dtype: float64


In [28]:
# !pip install rouge
from rouge import Rouge
rouge_scorer = Rouge()
r = df.iloc[10]
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
print(scores)

{'rouge-1': {'r': 0.45454545454545453, 'p': 0.45454545454545453, 'f': 0.45454544954545456}, 'rouge-2': {'r': 0.21621621621621623, 'p': 0.21621621621621623, 'f': 0.21621621121621637}, 'rouge-l': {'r': 0.3939393939393939, 'p': 0.3939393939393939, 'f': 0.393939388939394}}


In [34]:
print((scores['rouge-1']['f']+scores['rouge-2']['f']+scores['rouge-l']['f'])/3)

0.35490034990035496


In [ ]:
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

In [35]:
rouge_2_list = []
for i in range(df.shape[0]):
    r = df.iloc[i]
    scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
    rouge_2 = scores['rouge-2']['f']
    rouge_2_list.append(rouge_2)
np.mean(rouge_2_list)

0.20696501983423318